In [ ]:
from bert_score import BERTScorer
import numpy as np

# BERTScore explain and examples
# Ref: https://medium.com/@abonia/bertscore-explained-in-5-minutes-0b98553bfb71


def calculate_score(candidate_text, reference_text):
    # BERTScore calculation
    scorer = BERTScorer(model_type="bert-base-uncased")

    scores = []
    for candidate in [c.strip() for c in candidate_text.strip().split(".")]:
        if candidate == "":
            continue
        best_score = 0

        for reference in [c.strip() for c in reference_text.strip().split(".")]:
            if reference == "":
                continue

            P, R, F1 = scorer.score(cands=[candidate], refs=[reference])

            if F1 > best_score:
                best_score = F1

        scores.append(best_score)

    return np.mean(scores)

In [ ]:
candidate_full_list = [
    "The participant's commuter behavior is characterized by walking, driven by cost, health, and dog exercise. They don't own a car and prioritize walking over public transport for short distances. Shopping trips are frequent and spontaneous, influenced by proximity to the train station and local shops. They are moderately concerned about environmental issues, linking them to poverty and health. They express a desire to be more environmentally friendly but acknowledge cost as a barrier. They are happy with recycling ethos in their neighborhood but concerned about litter.",
]
reference_full_list = [
    "The participant is aware of environmental issues, particularly regarding plastic bag usage. The participant has mixed feelings about plastic bags, acknowledging their utility and reusability while also recognizing their negative environmental impact. The participant is annoyed by the plastic bag charge but understands its purpose in promoting more sustainable behavior. The participant believes the charge will encourage better organization and planning regarding shopping bags. The participant is concerned about the end use of the money collected from the charge. The participant is moderately concerned about environmental issues, linking them to poverty and health.",
]

for i in range(len(candidate_full_list)):
    print(
        f"Round {i+1} - F1 Mean: {calculate_score(candidate_full_list[i], reference_full_list[i]):.4f}"
    )

Round 1 - F1 Mean: 0.5854


In [9]:
from bert_score import BERTScorer

scorer = BERTScorer(model_type="bert-base-uncased")
print(scorer.score(cands=['Perceived Convenience'], refs=['personal habit']))
print(scorer.score(cands=['Perceived Convenience'], refs=['convenience']))


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


(tensor([0.6988]), tensor([0.6988]), tensor([0.6988]))
(tensor([0.6045]), tensor([0.7501]), tensor([0.6695]))


In [14]:
import numpy as np
from bert_score import BERTScorer
from tabulate import tabulate
import pandas as pd

from models.response_models import ScopeComponent


def display(pair_item):
    return f"{pair_item[0]} ({ScopeComponent.get_component_name_from_key(pair_item[1])})"

def calculate_best_code_pairs(reference_codes, candidate_codes, compare_key):
    scorer = BERTScorer(model_type="bert-base-uncased")

    reference_items = []
    for key, item in reference_codes.items():
        if key == "file":
            continue

        for item in item:
            reference_items.append((item[compare_key], key))

    # print(reference_items)

    candidate_items = []
    for key, item in candidate_codes.items():
        if key == "file":
            continue

        for item in item:
            candidate_items.append((item[compare_key], key))

    # print(candidate_items)


    best_pairs = []

    for reference_item in reference_items[:]:
        best_score = 0
        best_can_item = None
        for canidate_item in candidate_items:
            if reference_item[1] != canidate_item[1]:
                continue

            P, R, F1 = scorer.score(
                cands=[reference_item[0]], refs=[canidate_item[0]]
            )

            if F1 > best_score:
                best_score = F1
                best_can_item = canidate_item

        best_pairs.append([reference_item, best_can_item, round(float(best_score), 4)])

    table = []
    for pair in best_pairs:
        table.append((display(pair[0]), display(pair[1]), pair[2]))

    print(
        tabulate(
            table,
            headers=["Candidate Code", "Best Reference Code", "Best Score"],
            tablefmt="rst",
        )
    )
    print()
    
    pairs = [pair[2] for pair in best_pairs]
    similar_pair = [pair for pair in pairs if pair >= 0.75]

    print(f'Total codes: {len(pairs)}')
    print(f'Total similar codes (score >= 0.75): {len(similar_pair)}')
    print(f'Mean score: {np.mean(pairs)}')
    print()

    return best_pairs


In [15]:
import json

# Sentiment Analysis: Thematic analysis
with open("./travel/02_thematic_analysis_codes_human.txt") as f:
    reference_codes_raw = f.read().strip().split("\n\n")

with open("./travel/02_thematic_analysis_codes.txt") as f:
    candidate_codes_raw = f.read().strip().split("\n\n")

best_pairs = []
for i in range(len(candidate_codes_raw)):
    reference_codes: dict = json.loads(reference_codes_raw[i])
    candidate_codes: dict = json.loads(candidate_codes_raw[i])

    print(f'File: {candidate_codes['file']}')
    best_pairs += calculate_best_code_pairs(reference_codes, candidate_codes, "code")




# Overall aspect (data from all documents)
pairs = [pair[2] for pair in best_pairs]
similar_pair = [pair for pair in pairs if pair >= 0.75]

print("-"*50)
print(f'Overall Total Code: {len(pairs)}')
print(f'Overall Total similar codes (score >= 0.75): {len(similar_pair)}')
print(f'Overall Mean Similarity: {np.mean(pairs):.4f}')
print()

# Array of scores of each component
score_dict = {}

for pair in best_pairs:
    component_key = pair[0][1]
    score = pair[2]
    if component_key not in score_dict:
        score_dict[component_key] = []

    score_dict[component_key].append(score)

for key in  ScopeComponent.get_component_keys():
    print('{:<25}: {:.4f} ± {:.4f}'.format(ScopeComponent.get_component_name_from_key(key), np.mean(score_dict[key]), np.std(score_dict[key])))
print()

File: data/travel_scope_txt/Stage3_Crediton St Lwrence.txt
=====================================================  ===================================================  ============
Candidate Code                                         Best Reference Code                                    Best Score
=====================================================  ===================================================  ============
cyclists (Actors)                                      Residents Commuters (Actors)                               0.5275
tax payers  (Actors)                                   Residents Commuters (Actors)                               0.5725
residents  (Actors)                                    Residents Commuters (Actors)                               0.7156
pedestrians  (Actors)                                  Residents Commuters (Actors)                               0.571
cars (Physical Components)                             Alternative Vehicles (Physical Component

In [16]:
import json

# Sentiment Analysis: EABSS Scope
with open("./travel/02_eabss_scope_human.txt") as f:
    reference_codes_raw = f.read().strip().split("\n\n")[0]

with open("./travel/02_eabss_scope.txt") as f:
    candidate_codes_raw = f.read().strip().split("\n\n")[0]


reference_codes: dict = json.loads(reference_codes_raw)
candidate_codes: dict = json.loads(candidate_codes_raw)

best_pairs = calculate_best_code_pairs(reference_codes, candidate_codes, "element")

=======================================================================  ========================================================  ============
Candidate Code                                                           Best Reference Code                                         Best Score
=======================================================================  ========================================================  ============
Residents (Actors)                                                       Residents Commuters (Actors)                                    0.7156
Walking (Physical Components)                                            Cycling (Physical Components)                                   0.6143
Cycling (Physical Components)                                            Cycling (Physical Components)                                   1
Cars (Physical Components)                                               Cars (Physical Components)                                      1
Pu